# Homework 3

## FINM 37500: Fixed Income Derivatives

### Mark Hendricks

#### Winter 2024

# 1. Modeling the Volatility Smile

## Swaption Vol Data

The file `data/swaption_vol_data_2024-02-20.xlsx` has market data on the implied volatility skews for swaptions. Note that it has several columns:
* `expry`: expiration of the swaption
* `tenor`: tenor of the underlying swap
* `model`: the model by which the volatility is quoted. (All are Black.)
* `-200`, `-100`, etc.: The strike listed as difference from ATM strike (bps). Note that ATM is considered to be the **forward swapa rate** which you can calculate.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from scipy.stats import norm
from scipy.optimize import minimize

def Blacks_formula(sigma, maturity, strike, F, discount, price='Call'):
    vol=sigma
    T=maturity
    K=strike
    Z=discount
    d1=(np.log(F/K)+(((vol**2)*T)/2))/(vol*(T**(1/2)))
    d2=d1-(vol*(T**(1/2)))
    if price=='Call':
        return Z*((F*norm.cdf(d1))-(K*norm.cdf(d2)))
    if price=='Put':
        return Z*((K*norm.cdf(-d2))-(F*norm.cdf(-d1)))

def price_cap_floor(sigma, maturity, strike, F, discount, instrument='Cap', n=4):
    if instrument=='Cap':
        price='Call'
    if instrument == 'Floor':
        price='Put'
    tp=0
    for i in range(2,int(n*maturity)+1):
        tp+=Blacks_formula(sigma, (i/n)-(1/n), strike, F[i-1], discount[i-1], price)
    return (100/n)*tp

def forward_vols_from_instrument_prices(cap_price, maturity, strike, F, discount, price='Call', n=4):
    if np.isnan(cap_price):
        return np.nan
    def helper(sigma):
        return ((Blacks_formula(sigma, maturity, strike, F, discount, price)*100/n) - (cap_price))**2
    return minimize(helper,[0.05], tol=1e-8).x[0]

def estimate_theta(discounts,sigmas,n=4):
    if np.isnan(sigmas[0]):
        sigmas[0]=sigmas[1]
    discounts*=100
    thetas=np.zeros((len(sigmas)))
    tdis=0
    for i in range(len(sigmas)):
        theta=discounts

def continuous_from_discount(discount, maturity):
    '''Returns the continuous rate from the discount factor and the maturity'''
    return -np.log(discount)/maturity

def compounded_from_discount(discount, maturity, freq=2):
    '''Returns the compounded rate from the discount factor and the maturity'''
    return ((1/discount)**(1/(maturity*freq))-1)*freq

def discount_from_continuous(continuous, maturity):
    '''Returns the discount factor from the continuous rate and the maturity'''
    return np.exp(-continuous*maturity)

def discount_from_compounded(compounded, maturity, freq=2):
    '''Returns the discount factor from the compounded rate and the maturity'''
    return (1+compounded/freq)**(-maturity*freq)

def display_tree(periods, levels, attribute):
    temp=pd.DataFrame()
    for i in range(1,len(levels)+1):
        temp_level=[]
        for j in range(len(levels[-i])):
            temp_level.append(getattr(levels[-i][j],attribute))
        temp[round(periods[-i],2)]=temp_level+['']*(len(levels[-1])-len(temp_level))
    temp.sort_index(axis=1, inplace=True)
    temp.columns=temp.columns.astype(str)
    return temp.style.format("{:.2}")
        
def display_tree_no_attribute(periods, levels):
    temp=pd.DataFrame()
    for i in range(1,len(levels)+1):
        temp_level=[]
        for j in range(len(levels[-i])):
            temp_level.append(levels[-i][j])
        temp[round(periods[-i],2)]=temp_level+['']*(len(levels[-1])-len(temp_level))
    temp.sort_index(axis=1, inplace=True)
    temp.columns=temp.columns.astype(str)
    return temp
    
class tree_convention:
    UP=1
    DOWN=-1
    ORIGIN=0
    TERMINAL=2
    DUMMY=3
    
class interest_node():
    def __init__(self, period, rate, state=tree_convention.DUMMY):
        self.period = period
        self.rate = rate
        self.state = state

    def __repr__(self):
        return 'rate: {:.2%}\nperiod: {}\nstate: {}'.format(self.rate,self.period,self.state)
    
class Interest_Tree():
    def __init__(self, periods, rates=None):
        self.periods = periods
        self.rates = rates
        self.levels = []

    def tree_from_discounts_theta_sigma(self,discounts,theta,sigma,freq=4):
        self.levels.append([interest_node(0,continuous_from_discount(discounts[0],1/freq),tree_convention.ORIGIN)])
        for i in range(1,len(self.periods)):
            tlevel=[]
            for j in range(len(self.levels[i-1])):
                trate=self.levels[i-1][j].rate
                tlrate=np.log(trate*100)
                ttheta=theta[i-1]
                tsigma=sigma[i-1]
                tlrateup=tlrate+(ttheta*(1/freq))+(tsigma*np.sqrt(1/freq))
                trateup=np.exp(tlrateup)/100                   
                node=interest_node(self.periods[i],trateup)
                tlevel.append(node)
                if j == i-1:
                    tlratedown=tlrate+(ttheta*(1/freq))-(tsigma*np.sqrt(1/freq))
                    tratedown=np.exp(tlratedown)/100
                    node=interest_node(self.periods[i],tratedown)
                    tlevel.append(node)
            self.levels.append(tlevel)

    def __repr__(self):
        tree_str = 'Rates_Tree [%]:\n'
        for level_index, level in enumerate(self.levels):
            level_str = '  ' * level_index
            for node in level:
                node_str = '{:.2f}'.format(
                    node.rate*100)
                level_str += node_str + ' '
            tree_str += 'period: {} ||'.format(node.period) + level_str.strip() + '\n'
        return tree_str.strip()
                

class bond_node():
    def __init__(self, period, price, state, maturity,expected_price=None):
        self.period = period
        self.price = price
        self.state = state
        self.maturity = maturity
        self.expected_price=expected_price
        self.rn_probUP=1/2
        self.rn_probDOWN=1/2

    def __repr__(self):
        return 'period: {}\nstate: {}\nprice: {:.2f}\nmaturity: {}'.format(self.period, self.state, self.price, self.maturity)

class Binary_Bond_Tree():
    def __init__(self, periods=None, price=None, face_value=100, interest_tree=None, maturity=None):
        self.periods = np.array(periods)
        if len(price)<=1:
            self.price=price
        else:
            self.price=np.array(price)
        self.face_value = face_value
        self.levels = []
        self.interest_tree = interest_tree
        self.maturity = maturity
        self.thetas=[]
        self.sigmas=np.zeros(())

    def tree_from_sigmas(self,sigmas,periods=None, prices=None):
        if periods is not None:
            self.periods=periods
        freq=1/(self.periods[1]-self.periods[0])
        if prices is not None:
            self.price=prices
        self.sigmas=sigmas
        for i in range(len(self.periods)-2):
            def helper(theta):
                ttree=Interest_Tree(self.periods[:i+2])
                ttree.tree_from_discounts_theta_sigma(self.price[:i+2]/100,np.append(self.thetas[:i],theta),sigmas[:i+2],freq)
                tjprices=[]
                for j in range(len(ttree.levels[-1])):
                    tjprices.append(self.face_value*np.exp(-ttree.levels[-1][j].rate*(1/freq)))
                for j in range(2,len(ttree.levels)+1):
                    tkprices=[]
                    for k in range(len(ttree.levels[-j])):
                        average=(tjprices[k]+tjprices[k+1])/2
                        fact=np.exp(-ttree.levels[-j][k].rate*(1/freq))
                        tkprice=average*fact
                        tkprices.append(tkprice)
                    tjprices=tkprices
                if len(tjprices)!=1:
                    print('Error', len(tjprices))
                return (tjprices[0]-self.price[i+1])**2
            theta=minimize(helper,[0.05], tol=1e-8).x[0]
            self.thetas.append(theta)
        ttree=Interest_Tree(self.periods[:-1])
        ttree.tree_from_discounts_theta_sigma(self.price/100,self.thetas,sigmas,freq)
        self.interest_tree=ttree

    def __repr__(self):
        tree_str = 'Binary_Bond_tree:\n'
        tree_str += '                               Maturity: {}\n'.format(self.maturity)
        for level_index, level in enumerate(self.levels[:-1]):
            level_str = '  ' * level_index
            for node in level:
                node_str = '[state: {}, price: {:.2f}]'.format(
                    node.state, node.price)
                level_str += node_str + '  '
            tree_str += 'period: {} '.format(node.period) + level_str.strip() + '\n'
        return tree_str.strip()
    

In [4]:
data=pd.read_excel('../data/swaption_vol_data.xlsx')
data

,reference,instrument,model,date,expiration,tenor,-200,-100,-50,-25,0,25,50,100,200
0,SOFR,swaption,black,2024-02-20,1,4,54.54,40.37,35.94,34.23,32.83,31.71,30.86,29.83,29.54


Your data: ywill use a single row of this data for the `1x4` swaption.
* date: `2024-02-20`
* expiration: 1yr
* tenor: 4yrs

## Rate Data

The file `data/cap_quotes_2024-02-20.xlsx` gives 
* SOFR swap rates, 
* their associated discount factors
* their associated forward interest rates.

You will not need the cap data (flat or forward vols) for this problem.
* This cap data would be helpful in calibrating a binomial tree, but this problem focuses on Black's formula and SABR.

In [6]:
rates_cap=pd.read_excel('../data/cap_quotes_2024-02-20.xlsx',sheet_name='cap')
rates_sofr=pd.read_excel('../data/cap_quotes_2024-02-20.xlsx',sheet_name='sofr')

## The Swaption

Consider the following swaption with the following features:
* underlying is a fixed-for-floating (SOFR) swap
* the underlying swap has **quarterly** payment frequency
* this is a **payer** swaption, which gives the holder the option to **pay** the fixed swap rate and receive SOFR.

## 1.1
Calculate the (relevant) forward swap rate. That is, the one-year forward 4-year swap rate.


In [30]:
rates_sofr

,date,USOSFRC Curncy,USOSFRF Curncy,USOSFRI Curncy,USOSFR1 Curncy,USOSFR1C Curncy,USOSFR1F Curncy,USOSFR1I Curncy,USOSFR2 Curncy,USOSFR2C Curncy,...,USOSFR4F Curncy,USOSFR5 Curncy,USOSFR6 Curncy,USOSFR7 Curncy,USOSFR8 Curncy,USOSFR9 Curncy,USOSFR10F Curncy,USOSFR10 Curncy,USOSFR10C Curncy,USOSFR10I Curncy
0,maturity,0.246407,0.498289,0.750171,1.002053,1.245722,1.497604,1.749487,2.001369,2.245038,...,4.498289,5.002053,6.001369,7.000684,8.0000,9.002053,10.001369,10.001369,10.001369,10.001369
1,2022-01-03 00:00:00,0.090700,0.200000,0.306700,0.406000,0.497000,0.592000,0.679500,0.762900,0.832500,...,1.155000,1.202200,1.263600,1.314900,1.3568,1.390900,1.423700,1.423700,1.423700,1.423700
2,2022-01-04 00:00:00,0.090800,0.197400,0.298100,0.394900,0.487500,0.580700,0.667000,0.751500,0.821500,...,1.156000,1.199600,1.266300,1.322700,1.3668,1.402800,1.437000,1.437000,1.437000,1.437000
3,2022-01-05 00:00:00,0.098200,0.218000,0.331300,0.440000,0.539000,0.639500,0.735500,0.820300,0.893000,...,1.229500,1.274600,1.341000,1.394300,1.4348,1.467800,1.499400,1.499400,1.499400,1.499400
4,2022-01-06 00:00:00,0.111700,0.239000,0.356900,0.463000,0.565500,0.667000,0.765000,0.853000,0.927000,...,1.274500,1.316100,1.379400,1.428600,1.4654,1.494600,1.524000,1.524000,1.524000,1.524000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
550,2024-02-14 00:00:00,5.322500,5.248400,5.133300,5.002700,4.832800,4.682100,4.564000,4.468600,4.372000,...,4.017000,3.989400,3.948700,3.924100,3.9098,3.903200,3.900000,3.900000,3.900000,3.900000
551,2024-02-15 00:00:00,5.316400,5.239400,5.128400,4.998200,4.823700,4.675600,4.553900,4.456500,4.359000,...,3.996000,3.968200,3.926100,3.900600,3.8860,3.879100,3.876200,3.876200,3.876200,3.876200
552,2024-02-16 00:00:00,5.322500,5.257500,5.154600,5.032200,4.865400,4.723400,4.605100,4.509000,4.415500,...,4.050000,4.021300,3.975500,3.948200,3.9314,3.922000,3.917100,3.917100,3.917100,3.917100
553,2024-02-19 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,4.612400,NaN,NaN,...,NaN,NaN,3.981000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
trates=rates_sofr[rates_sofr['date']==pd.to_datetime('2024-02-20')]
trates=trates.iloc[0,1:].values
tmaturities=rates_sofr.iloc[0]
tmaturities=tmaturities[1:].values

In [29]:
trates

array([5.3248, 5.2551, 5.147, 5.019, 4.8517, 4.7056, 4.5828, 4.4863,
       4.3895, 4.315, 4.26, 4.2192, 4.132, 4.0807, 4.0295, 4.0018, 3.9613,
       3.9372, 3.9232, 3.9168, 3.9146, 3.9146, 3.9146, 3.9146],
      dtype=object)

In [27]:
discounts=


## 1.2
Price the swaptions at the quoted implied volatilites and corresponding strikes, all using the just-calculated forward swap rate as the underlying.



## 1.3
To consider how the expiration and tenor matter, calculate the prices of a few other swaptions for comparison. 
* No need to get other implied vol quotes--just use the ATM implied vol you have for the 1x2 above. (Here we are just interested in how Black's formula changes with changes in tenor and expiration.
* No need to calculate for all the strikes--just do the ATM strike.

Alternate swaptions
* The 3mo x 4yr swaption
* The 2yr x 4yr swaption
* the 1yr x 2yr swaption

Report these values and compare them to the price of the `1y x 4y` swaption.

***

# 2. Pricing w/ BDT

Use the data in `cap_curves_2024-02-20.xlsx`.

## 2.1

Calibrate the BDT Tree
* theta to fit the term structure discounts.
* sigma to fit the fwd vols from the cap data.

Report the rate tree through $T=5$. Report trees for rates compounded
* continuously
* annually

## 2.2

Use a tree to price a vanilla fixed-rate, 5-year bond with coupon rate equal to the forward swap rate calculated in problem `1.1.`

## 2.3

We will calculate the binomial tree for the 5-year swap, but here we do so by valuing the swap as...

$$\text{payer swap} = \text{floating rate note} - \text{fixed-rate bond}$$

Recall for the Floating-Rate Note:
* It has par value of 100 at each reset date.
* Every node is a reset date given the assumptions of the swap timing.

Report the tree for the 5-year swap.

## 2.4



Report the binomial tree for the one-year swaption on a 4-year swap with **european** exercise.
* At expiration, the swap tree from 2.3 will have 4 years left, as desired for pricing the 1y-4y swaption.

## 2.5

Compare the pricing of the 1y4y swaption from Black's formula in Section 1 vs the binomial tree.

## 2.6

Reprice the swaption using the BDT tree, but this time assuming it is **american**-style exercise.

***

# 3. Midcurve Swaptions

## 3.1 

Use the BDT tree from section 2 to price a **european** midcurve swaption 1y$\rightarrow$2y$\rightarrow$2y.

## 3.2

Price the **american** midcurve swaption 1y$\rightarrow$2y$\rightarrow$2y.

***